In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

# OpenAI

In [2]:
from nemoguardrails import LLMRails, RailsConfig

# initialize rails config
config = RailsConfig.from_path("./fact_check_config/openai")

# create rails
app = LLMRails(config, verbose=True)

/Users/sofiaperez/anaconda3/envs/llms/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
history = [{"role": "user", "content": "What was the unemployment rate in March 2023?"}]
bot_message = await app.generate_async(messages=history)
print(bot_message['content'])

According to the US Bureau of Labor Statistics, the unemployment rate in March 2023 was 3.5 percent.


In [4]:
history.append(bot_message)
history.append(
    {"role": "user", "content": "What was the unemployment rate for teenagers?"}
)
bot_message = await app.generate_async(messages=history)
print(bot_message['content'])

According to the US Bureau of Labor Statistics, the unemployment rate for teenagers in March 2023 was 9.8 percent.


In [5]:
history.append(bot_message)
history.append(
    {"role": "user", "content": "What was the unemployment rate for senior citizens?"}
)
bot_message = await app.generate_async(messages=history)
print(bot_message['content'])


According to the US Bureau of Labor Statistics, the unemployment rate for senior citizens in March 2023 was 4.7 percent.


## Adding the fact check rail

In [6]:
# initialize rails config
config = RailsConfig.from_path("./fact_check_config/openai")

# create rails
app = LLMRails(config, verbose=True)

In [7]:
history = [{"role": "user", "content": "What was the unemployment rate in March 2023?"}]
bot_message = await app.generate_async(messages=history)
print(bot_message['content'])

According to the March 2023 US jobs report, the unemployment rate in March 2023 was 3.5 percent.


In [8]:
history.append(bot_message)
history.append(
    {"role": "user", "content": "What was the unemployment rate for teenagers?"}
)
bot_message = await app.generate_async(messages=history)
print(bot_message['content'])

According to the March 2023 US jobs report, the unemployment rate for teenagers was 9.8 percent.


In [9]:
history.append(bot_message)
history.append(
    {"role": "user", "content": "What was the unemployment rate for senior citizens?"}
)
bot_message = await app.generate_async(messages=history)
print(bot_message['content'])

I don't have enough information to answer


# Llama2

In [4]:
import huggingface_hub
huggingface_hub.login(token=os.environ["HUGGINGFACEHUB_API_TOKEN"])


from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

pipe = pipeline(
    "text-generation",
    model=model, 
    tokenizer=tokenizer,
    max_length=4096,
    do_sample=True,
    temperature=0.1,
    top_p=0.95,
    logprobs=None,
    top_k=40,
    repetition_penalty=1.1
)

llm = HuggingFacePipeline(pipeline=pipe)

/Users/sofiaperez/anaconda3/envs/llms/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/sofiaperez/.cache/huggingface/token
Login successful


Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.86s/it]


In [11]:
from nemoguardrails.llm.helpers import get_llm_instance_wrapper
from nemoguardrails.llm.providers import register_llm_provider
from nemoguardrails import LLMRails, RailsConfig

HFPipeline = get_llm_instance_wrapper(
    llm_instance=llm, llm_type="hf_pipeline_llama2"
)

register_llm_provider("hf_pipeline_llama2", HFPipeline)

# initialize rails config
config = RailsConfig.from_path("./fact_check_config/llama2")

# create rails
app = LLMRails(config, verbose = True)

In [12]:
await app.generate_async(prompt="What was the unemployment rate in March 2023?")

Parameter temperature does not exist for WrapperLLM


Parameter temperature does not exist for WrapperLLM


'According to the latest US jobs report, the unemployment rate in March 2023 was 3.5 percent.'

In [13]:
await app.generate_async(prompt="What was the unemployment rate for senior citizens?")

Parameter temperature does not exist for WrapperLLM


Parameter temperature does not exist for WrapperLLM


'According to the most recent data from the Bureau of Labor Statistics, the unemployment rate for seniors (aged 65 and older) was 3.2% in March 2023. However, I must inform you that this figure may not be accurate due to various factors such as underreporting or misclassification of workers. If you have any further questions or concerns, please feel free to ask.'

## Adding the fact check rail

In [20]:
from nemoguardrails import LLMRails, RailsConfig

# initialize rails config
config = RailsConfig.from_path("./fact_check_config/llama2")

# create rails
app = LLMRails(config, verbose = True)

In [21]:
await app.generate_async(prompt="What was the unemployment rate in March 2023?")

Parameter temperature does not exist for WrapperLLM


Parameter temperature does not exist for WrapperLLM


''

## Adding the fact check rail + few shot in prompt

In [5]:
from nemoguardrails.llm.helpers import get_llm_instance_wrapper
from nemoguardrails.llm.providers import register_llm_provider
from nemoguardrails import LLMRails, RailsConfig

HFPipeline = get_llm_instance_wrapper(
    llm_instance=llm, llm_type="hf_pipeline_llama2"
)

register_llm_provider("hf_pipeline_llama2", HFPipeline)

# initialize rails config
config = RailsConfig.from_path("./fact_check_config/llama2")

# create rails
app = LLMRails(config, verbose = True)

In [6]:
await app.generate_async(prompt="What was the unemployment rate for senior citizens?")

Parameter temperature does not exist for WrapperLLM


Parameter temperature does not exist for WrapperLLM


"I don't have enough information to answer"